In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [47]:
paper_pmid = 11701889
paper_name = 'ooi_boeke_2001' 

In [48]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [49]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [50]:
original_data = pd.read_excel('raw_data/ooi_boeke_2001.xlsx', sheet_name='Sheet1', header=None)

In [51]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 29 x 3


In [52]:
original_data.head()

,0,1,2
0,YKU80,+,+
1,LIG4,-,+
2,SIR2,-,+
3,LIF1,+,+
4,NEJ1,+,+


In [53]:
original_data['gene'] = original_data[0].astype(str)

In [54]:
# Eliminate all white spaces & capitalize
original_data['gene'] = clean_genename(original_data['gene'])

In [55]:
original_data.loc[original_data['gene']=='GPE2','gene'] = 'GPB2'

In [56]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['gene'], to='orf')

In [57]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [0, 1, 2, gene, orf]
Index: []


In [58]:
data_switch = {'-': 0, '+': -1}
for c in [1,2]:
    original_data[c] = original_data[c].apply(lambda x: data_switch[x])

In [59]:
original_data.set_index('orf', inplace=True)

In [60]:
original_data = original_data[[1,2]].copy()

In [61]:
original_data = original_data.groupby(original_data.index).mean()

In [62]:
original_data.shape

(29, 2)

# Prepare the final dataset

In [63]:
data = original_data.copy()

In [64]:
dataset_ids = [315, 2]
datasets = datasets.reindex(index=dataset_ids)

In [65]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [66]:
data.head()

dataset_id,315,2
data_type,value,value
orf,,
YAL056W,-1,-1
YBL021C,-1,0
YCL033C,-1,-1
YCR050C,-1,-1
YDL042C,0,-1


## Subset to the genes currently in SGD

In [67]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [68]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,315,2
,data_type,value,value
gene_id,orf,,
52,YAL056W,-1,-1
109,YBL021C,-1,0
520,YCL033C,-1,-1
607,YCR050C,-1,-1
1920,YDL042C,0,-1


# Normalize

In [69]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [70]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [71]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,315,2,315,2
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
52,YAL056W,-1,-1,-14.387495,-12.753151
109,YBL021C,-1,0,-14.387495,0.000000
520,YCL033C,-1,-1,-14.387495,-12.753151
607,YCR050C,-1,-1,-14.387495,-12.753151
1920,YDL042C,0,-1,0.000000,-12.753151


# Print out

In [72]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [73]:
from IO.save_data_to_db3 import *

In [74]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 11701889...
Inserting the new data...


100%|██████████| 2/2 [00:00<00:00,  8.42it/s]

Updating the data_modified_on field...
